In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import glob
from tqdm.notebook import tqdm
import gc

In [ ]:
df_train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
df_train


In [ ]:
df_train.head()

In [ ]:
df_train.head(10)

In [ ]:
df_test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
df_test


In [ ]:
df_test.head()

In [ ]:
submission = pd.read_csv('../input/optiver-realized-volatility-prediction/sample_submission.csv')
submission

In [ ]:
submission.head()

In [ ]:
def WAP1(df):
    WAP = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * 
           df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
    return WAP

def WAP2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * 
           df['bid_size2'])/(df['bid_size2'] + df['ask_size2'])
    return wap

def log_return(WAP):
    return np.log(WAP).diff() 

def realized_volatility(log_r):
    return np.sqrt((log_r**2).sum())

In [ ]:
df_train.describe()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train_final = df_train.replace([np.inf,-np.inf],np.nan).dropna()
df_train_final[df_train_final.isnull().any(axis=1)]
df_train_final

In [ ]:
df_train_final.corr()['target']

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
sns.heatmap(df_train_final.corr(),annot=True,ax=ax)

In [ ]:
sns.scatterplot(x=df_train_final['stock_id'],y=df_train_final['target'])

In [ ]:
sns.scatterplot(x=df_train_final['time_id'],y=df_train_final['target'])

In [ ]:
from sklearn.metrics import r2_score
def rmspe_R_squared(y_true, y_pred):
    return  ((np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))),r2_score(y_true, y_pred))

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10,6)

In [ ]:
plt.hist(df_train.target, bins=20, rwidth=0.8)
plt.xlabel('target')
plt.ylabel('Count')
plt.show()

In [ ]:

from scipy.stats import norm
import numpy as np
plt.hist(df_train.target, bins=20, rwidth=0.8, density=True)
plt.xlabel('target')
plt.ylabel('Count')

rng = np.arange(df_train.target.min(), df_train.target.max(), 0.1)
plt.plot(rng, norm.pdf(rng,df_train.target.mean(),df_train.target.std()))

In [ ]:
upper_limit = df_train.target.mean() + 4*df_train.target.std()
upper_limit

In [ ]:
lower_limit = df_train.target.mean() -4*df_train.target.std()
lower_limit

In [ ]:
df_train[(df_train.target>upper_limit) | (df_train.target<lower_limit)]

In [ ]:
Q1 = df_train.target.quantile(0.45)
Q3 = df_train.target.quantile(0.55)
Q1, Q3

In [ ]:
df_train.shape

In [ ]:
IQR = Q3 - Q1
IQR

In [ ]:
lower_limit = Q1 - 1.5*IQR
upper_limit = Q3 + 1.5*IQR
lower_limit, upper_limit

In [ ]:
from sklearn.metrics import mean_absolute_error
def mean(y,predit):
    mae=mean_absolute_error(y,predit)
    return(mae)